### [인덱스 다루기 - MultiIndex]
- 기본 인덱스
    * .set_index() <- 컬럼을 인덱스로 설정
    * .reindex() <- 인덱스 재구성(일부 변경. 결측치 발생 가능)
    * .reset_index() <- 위치인덱스(RangeIndex)로 변경. 기존 인덱스는 컬럼 추가
    * .iloc[위치인덱스] <- 판다스에서 DataFrame/Series에 데이터가 존재하는 위치 번호를 지정/정수
    * .loc[라벨인덱스] <- DataFrame/Series 생성 시 index, columns 매개변수로 지정한 인덱스들
- 멀티 인덱스
    * .xs() <- 인덱스 레벨에 따른 데이터 선택하는 메서드

[1] 모듈 로딩 및 데이터 준비 <hr>

In [221]:
# 모듈 로딩
import pandas as pd

In [222]:
# 데이터 준비
FILE_NAME = '../DATA/학생관리부.xlsx'

# index_col = int, str, sequence int, slicing
dataDF = pd.read_excel(FILE_NAME, skiprows=2, index_col=[1, 2]) # index_col에 시퀀스 쓰려면 int로

In [223]:
# 확인
dataDF.head()

이름   석차
학년 반          
1  1  홍길동    8
   3  고길동    1
   7  이길동   12
3  1  박길동    9
2  1  최길동   43

In [224]:
# ================================
# 멀티인덱스 속성 확인
# ================================
mIdx = dataDF.index

print('-----------------------------------------')
print(f'type(mIdx)  : {type(mIdx)}')
print(f'names       : {mIdx.names}')
print(f'levels      : {mIdx.levels}')
print(f'dtypes      : {mIdx.dtypes}')
print(f'nlevels     : {mIdx.nlevels}')
print('-----------------------------------------')
print(f'levshape    : {mIdx.levshape}')
print(mIdx)

-----------------------------------------
type(mIdx)  : <class 'pandas.core.indexes.multi.MultiIndex'>
names       : ['학년', '반']
levels      : [[1, 2, 3], [1, 3, 7]]
dtypes      : 학년    int64
반     int64
dtype: object
nlevels     : 2
-----------------------------------------
levshape    : (3, 3)
MultiIndex([(1, 1),
            (1, 3),
            (1, 7),
            (3, 1),
            (2, 1)],
           names=['학년', '반'])


[2] 데이터프레임에서 인덱스 관련 메서드 다루기 <hr>

In [225]:
# ====================================================
## [2-1] 컬럼을 인덱스로 설정하기: .set_index()
# ====================================================
dataDF2 = dataDF.set_index(['이름']).sort_index().copy()
dataDF2

,석차
이름,
고길동,1
박길동,9
이길동,12
최길동,43
홍길동,8


In [226]:
# [실습] 석차 컬럼을 행 인덱스로 설정, 내림차순으로 정렬해서 저장하기
dataDF3 = dataDF.set_index([' 석차']).sort_index(ascending=False)
# dataDF3.sort_index(ascending=True, inplace=True)
dataDF3

,이름
석차,
43,최길동
12,이길동
9,박길동
8,홍길동
1,고길동


In [227]:
# [매개변수 활용] 특정 컬럼이 행인덱스 가능한지 검사 진행
#               대량의 데이터의 경우 행 중복값 검사를 눈으로 하기 어려움
#               verify_integrity=True, 행 중복 체크
# dataDF3 = dataDF.set_index(['학년'], verify_integrity=True)

In [228]:
# [문제] 2개 이상의 컬럼을 조합해서 행인덱스로 설정된 데이터 프레임을 만들기
dataDF4 = dataDF.set_index(['이름', ' 석차'], verify_integrity=True)
dataDF4.sort_index(level=[' 석차', '이름'], ascending=[True, True], inplace=True)

dataDF4

,
이름,석차
고길동,1
홍길동,8
박길동,9
이길동,12
최길동,43


In [229]:
# ==========================================================
# [2-2] 인덱스 재구성/재배치: .reindex(index=[], columns=[])
#                          .reindex(index=[], axis=0 또는 1)
# ==========================================================
dataDF

이름   석차
학년 반          
1  1  홍길동    8
   3  고길동    1
   7  이길동   12
3  1  박길동    9
2  1  최길동   43

In [230]:
# 이름, 학년, 반, 석차 ==> 학년, 반, 이름, 석차로 열이름/열인덱스를 재배치/재구성
# dataDF.reindex(labels=['학년', '반', '이름', '석차'], axis=1)
dataDF.reindex(columns=['학년', '반', '이름', '석차'])

학년   반   이름  석차
학년 반                 
1  1 NaN NaN  홍길동 NaN
   3 NaN NaN  고길동 NaN
   7 NaN NaN  이길동 NaN
3  1 NaN NaN  박길동 NaN
2  1 NaN NaN  최길동 NaN

In [231]:
# 이름, 학년, 반, 석차 ==> 학교, 이름, 학년, 반, 번호   열이름/열인덱스를 재배치/재구성
# -> 기존 없던 것 추가: 학년, 번호
# -> 기존 있던 것 제거: 석차
# dataDF.reindex(labels=['학교', '이름', '학년', '반', '번호'], axis='columns')
dataDF.reindex(columns=['학교', '이름', '학년', '반', '번호'], fill_value='unknown') # 결측치 -> unknown으로

학교   이름       학년        반       번호
학년 반                                         
1  1  unknown  홍길동  unknown  unknown  unknown
   3  unknown  고길동  unknown  unknown  unknown
   7  unknown  이길동  unknown  unknown  unknown
3  1  unknown  박길동  unknown  unknown  unknown
2  1  unknown  최길동  unknown  unknown  unknown

In [232]:
# =====================================================
# [2-3] 인덱스 초기화: .reset_index()
#                    기존 행인덱스는 컬럼으로 추가됨
# =====================================================
display(dataDF)

# 행인덱스 초기화 => [기본값: 행인덱스 -> 컬럼 추가]
dataDF2 = dataDF.reset_index()
display(dataDF2)

이름   석차
학년 반          
1  1  홍길동    8
   3  고길동    1
   7  이길동   12
3  1  박길동    9
2  1  최길동   43

,학년,반,이름,석차
0,1,1,홍길동,8
1,1,3,고길동,1
2,1,7,이길동,12
3,3,1,박길동,9
4,2,1,최길동,43


In [233]:
# 행인덱스 초기화 => [설정: 행인덱스 -> 컬럼 추가X
dataDF2 = dataDF.reset_index(drop=True)
display(dataDF2)

,이름,석차
0,홍길동,8
1,고길동,1
2,이길동,12
3,박길동,9
4,최길동,43


In [234]:
# -----------------------------------------------
# [실습] 현재 행인덱스 초기화: 컬럼으로 전달되는 행인덱스 삭제 여부 결정
#                           필요없는 데이터 drop=True
#                           필요 있는 데이터 drop=False     <--- 기본값
# -----------------------------------------------
dataDF = pd.DataFrame([('bird', 389.0),
                        ('bird', 24.0),
                        ('mammal', 80.5),
                        ('mammal', pd.NA)],
                        index=['falcon', 'parrot', 'lion', 'monkey'],
                        columns=('class', 'max_speed'))
display(dataDF)

# 컬럼 추가된 경우 컬럼명 지정 매개변수: names=컬럼명
dataDF2 = dataDF.reset_index(names='kind')
display(dataDF2)

,class,max_speed
falcon,bird,389.0
parrot,bird,24.0
lion,mammal,80.5
monkey,mammal,<NA>


,kind,class,max_speed
0,falcon,bird,389.0
1,parrot,bird,24.0
2,lion,mammal,80.5
3,monkey,mammal,<NA>


In [235]:
dataDF3 = dataDF2.set_index('kind')
dataDF3

,class,max_speed
kind,,
falcon,bird,389.0
parrot,bird,24.0
lion,mammal,80.5
monkey,mammal,<NA>
